In [11]:
import calendar_utils
import json

In [12]:
import importlib 
importlib.reload(calendar_utils)

<module 'calendar_utils' from '/Users/hayley/Documents/conv_ai/calendar_utils.py'>

In [13]:
from dotenv import load_dotenv
import os
import re
import datetime

load_dotenv()

True

In [14]:
# Basic ChatGPT class
import openai
from openai import OpenAI


class BasicChatGPT:
  def __init__(self, openai_api_key, model ="gpt-4-1106-preview", json_output=False, max_tokens=100):
    openai.api_key = openai_api_key
    self.max_tokens = max_tokens
    self.messages = []
    self.client = OpenAI()
    self.json_output = json_output
    self.model = model

  def call(self):
    if self.json_output:
      response = self.client.chat.completions.create(
        model=self.model,
        response_format={ "type": "json_object" }, ###### !!!!!
        messages=self.messages,
        max_tokens=self.max_tokens
      )
    else:
      response = self.client.chat.completions.create(
        model=self.model,
        messages=self.messages,
        max_tokens=self.max_tokens
      )
    return response

  def prompt(self, text):
    # Update the messages so far
    self.messages.append({
        "role": "user",
        "content": text,
    })

    # Call ChatGPT
    response = self.call()

    # Save the returned message
    message = response["choices"][0]["message"]
    self.messages.append(message)

    return message["content"]


In [5]:
# # Run an interactive console to chat with ChatGPT
# def run_console(chatgpt):
#     while True:
#         # Receive user input
#         prompt = input("User: ")

#         # Exit the loop if the user entered "exit"
#         if prompt == "exit":
#             break

#         # Receive and display ChatGPT response
#         response = chatgpt.prompt(prompt)
#         print("ChatGPT:", response)
#     print("The chat has ended")

In [15]:
# Make a ChatGPT object
basic_chatgpt = BasicChatGPT(os.environ.get("OPENAI_API_KEY"), json_output=True, max_tokens = 120)

# run_console(basic_chatgpt)

In [16]:
# calendar service 정의하는 부분 -> credentials.json 이 경로에 있어야 하고 
# 그리고 팝업 뜨면 너가 authorize 해줘야함 그리고 너 노트북이나 데탑 로컬에서 해야 함
# https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/calendar_v3.events.html#list
service = calendar_utils.get_calendar_service()

In [8]:
## if user asks about "today's schedule"
## first detect today
## then find what's today
## then request calendar for today's schedule

In [9]:
# first detect today and then find what date is today

input_text = f""" Today is {datetime.datetime.today().date()}.
Detect any time-related phrase from the given input from the user and resolve it into a date in the format of YYYY/MM/DD.
Examples include today, tomorrow, this Wednesday, next Tuesday, last Friday, 11/13, November 5th, 13th of July.
When the detected word is day of week, make sure to include adjective in front of it such as "this", "next", "upcoming", "last", "past".
\n\n
Your output must be in this format. Json("detected_phrase": <detected phrase>, "date": <YYYY/MM/DD>)

\n\n
Example 1
Input: Today is 2023/1/2. ... I want to schedule a meeting at 5PM tomorrow.
Output: tomorrow, 2023/1/3

Example 2
Input: Today is 2023/1/2. ... What time does the class start next Friday?
Output: next Friday, 2023/1/13

Example 3
Input: Today is 2023/1/2. ... What is the date of this upcoming Friday?
Output: this upcoming Friday, 2023/1/6
\n\nInput: 
"""

input_text += "I want to see a list of schedule from Monday two weeks later."



basic_chatgpt.messages.append({
    "role": "user",
    "content": input_text,
})

# Call ChatGPT
response = basic_chatgpt.call()
message = response.choices[0].message.content
init_result = json.loads(message)

print(message)
# Save the returned message
# message = response["choices"][0]["message"]
# self.messages.append(message)



AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [102]:
# mydate = datetime.datetime.strptime(init_result['date'], "%Y/%m/%d").isoformat() + "Z"
# mydate_p1 = (datetime.datetime.strptime(init_result['date'], "%Y/%m/%d") + datetime.timedelta(days=1)).isoformat() + "Z"
mydate = datetime.datetime.now(datetime.UTC).isoformat().split('+')[0] + "Z"
mydate_p1 = (datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=1)).isoformat().split('+')[0] + "Z"

In [104]:
#https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/calendar_v3.events.html#list

calendar_utils.get_event_list_recent(timeMin=mydate, timeMax=mydate_p1)

2023-10-08T23:00:00+09:00 기도
2023-11-22 규인 휴가


[{'kind': 'calendar#event',
  'etag': '"3393470160096000"',
  'id': '_8os38chm650j2ba274qj2b9k70rj2b9p6cr48ba48p2j8da16spj6ci460',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=XzhvczM4Y2htNjUwajJiYTI3NHFqMmI5azcwcmoyYjlwNmNyNDhiYTQ4cDJqOGRhMTZzcGo2Y2k0NjBfMjAyMzEwMDhUMTQwMDAwWiBoYXlsZXlzb245MkBt',
  'created': '2023-10-08T03:17:49.000Z',
  'updated': '2023-10-08T03:18:00.048Z',
  'summary': '기도',
  'creator': {'email': 'hayleyson92@gmail.com', 'self': True},
  'organizer': {'email': 'hayleyson92@gmail.com', 'self': True},
  'start': {'dateTime': '2023-10-08T23:00:00+09:00', 'timeZone': 'Asia/Seoul'},
  'end': {'dateTime': '2023-10-09T00:00:00+09:00', 'timeZone': 'Asia/Seoul'},
  'recurrence': ['RRULE:FREQ=DAILY'],
  'iCalUID': 'F84261A1-B951-4871-936D-DFE45A7332D0',
  'sequence': 1,
  'reminders': {'useDefault': True},
  'eventType': 'default'},
 {'kind': 'calendar#event',
  'etag': '"3399772430664000"',
  'id': '_68pj6d2660pj4ba460p3ab9k6so3eba18gr

In [7]:
event = {
  'summary': 'Dancing in the moonlight',
  'location': 'LaLa Land',
  'description': 'I plan to dance with Ryan Gosling in the yellow dress.',
  'start': {
    'dateTime': '2023-11-28T09:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'end': {
    'dateTime': '2023-11-28T17:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'recurrence': [
    'RRULE:FREQ=DAILY;COUNT=2'
  ],
  'attendees': [
    {'email': 'ryangosling@example.com'}
  ],
  'reminders': {
    'useDefault': False,
    'overrides': [
      {'method': 'email', 'minutes': 24 * 60},
      {'method': 'popup', 'minutes': 10},
    ],
  },
}

In [8]:
event = service.events().insert(calendarId='primary', body=event).execute()
print('Event created: %s' % (event.get('htmlLink')))

Event created: https://www.google.com/calendar/event?eid=NWxjaDk2YXI2NHI2MTFwZmowNDB0b2JmdDhfMjAyMzExMjhUMTYwMDAwWiBoYXlsZXlzb245MkBt


In [32]:
# Make a ChatGPT object
basic_chatgpt = BasicChatGPT(os.environ.get("OPENAI_API_KEY"), json_output=True, max_tokens = 1000)


# first detect today and then find what date is today

input_text = """You are a sophisticated calendar management assistant, adept at organizing and managing calendar schedules for both simple and complex tasks. Your role involves integrating tasks into a user's calendar with precision and ensuring that all details are accurately reflected.

Example Agenda:
For a given day, check the user's Calendar and output the agenda for the day in markdown using relevant emojis as bullet points. Here's an example:

Tues. Nov. 7th:

Check-in at Hyatt Regency Seattle

⏰ After 4:00 PM PT
📍 The Location: Hyatt Regency, Seattle
Reid / Sheryl 1:1

⏰ 6:00 PM PT
👥 Sheryl Soo(sheryl@zapier.com), Mike Knoop (Knoop@zapier.com)
📍 Virtual
...

Task Handling:
Simple Task: For straightforward tasks like "add meeting with Michael tomorrow at 8 PM," directly add this event to the calendar with title at the specified date and time.
Generate a json file that can be used as a calendar create request. Make sure you always close open parenthesis. Make sure to create a valid JSON!!!!!
{
  'summary': 'Dancing in the moonlight',
  'location': 'LaLa Land',
  'description': 'I plan to dance with Ryan Gosling in the yellow dress.',
  'start': {
    'dateTime': '2023-11-28T09:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'end': {
    'dateTime': '2023-11-28T17:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'recurrence': [
    'RRULE:FREQ=DAILY;COUNT=2'
  ],
  'attendees': [
    {'email': 'ryangosling@example.com'}
  ]
}

Complex Task: For intricate tasks such as "I have a conference talk this Tuesday. Please plan for it." plan and add related preparatory tasks. 
For example, schedule 'prepare PowerPoint presentation' on Monday and 'practice talk' on Tuesday.

Additional Responsibilities:
When more information about a meeting or attendee is required, search the web for details such as news about the company or the attendee's professional background.
Inform the user that they must respond after an action is completed to proceed.
"""

input_text += f"Today is {datetime.datetime.today().date()}. Add a meeting with Ryan Gosling at 9AM tomorrow at Seoul National University. It's Korean Standard Time."



basic_chatgpt.messages.append({
    "role": "user",
    "content": input_text,
})

# Call ChatGPT
response = basic_chatgpt.call()
message = response.choices[0].message.content
# init_result = json.loads(message)

print(message)


{
  "summary": "Meeting with Ryan Gosling",
  "location": "Seoul National University, Seoul, South Korea",
  "description": "Discuss collaboration opportunities with Ryan Gosling.",
  "start": {
    "dateTime": "2023-11-22T09:00:00+09:00",
    "timeZone": "Asia/Seoul"
  },
  "end": {
    "dateTime": "2023-11-22T10:00:00+09:00",
    "timeZone": "Asia/Seoul"
  },
  "attendees": [
    {"email": "ryan.gosling@example.com"}
  ]
}


In [34]:
event = service.events().insert(calendarId='3669abef1c37b027d0f67733d06ea0dfdb30be790671aa54befc7679044ebebb@group.calendar.google.com', body=json.loads(message)).execute()

In [35]:
event

{'kind': 'calendar#event',
 'etag': '"3401143087584000"',
 'id': 'did9ts9t0msmh98dimn90p5fu0',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=ZGlkOXRzOXQwbXNtaDk4ZGltbjkwcDVmdTAgMzY2OWFiZWYxYzM3YjAyN2QwZjY3NzMzZDA2ZWEwZGZkYjMwYmU3OTA2NzFhYTU0YmVmYzc2NzkwNDRlYmViYkBn',
 'created': '2023-11-21T12:59:03.000Z',
 'updated': '2023-11-21T12:59:03.792Z',
 'summary': 'Meeting with Ryan Gosling',
 'description': 'Discuss collaboration opportunities with Ryan Gosling.',
 'location': 'Seoul National University, Seoul, South Korea',
 'creator': {'email': 'hayleyson92@gmail.com'},
 'organizer': {'email': '3669abef1c37b027d0f67733d06ea0dfdb30be790671aa54befc7679044ebebb@group.calendar.google.com',
  'displayName': 'Conv-ai',
  'self': True},
 'start': {'dateTime': '2023-11-22T09:00:00+09:00', 'timeZone': 'Asia/Seoul'},
 'end': {'dateTime': '2023-11-22T10:00:00+09:00', 'timeZone': 'Asia/Seoul'},
 'iCalUID': 'did9ts9t0msmh98dimn90p5fu0@google.com',
 'sequence': 0,
 'att

In [33]:
event = service.events().insert(calendarId='primary', body=json.loads(message)).execute()
print('Event created: %s' % (event.get('htmlLink')))

Event created: https://www.google.com/calendar/event?eid=dTF2ZmlpOTNkcnVsY3ZxbGlra3Q4bzVtbGsgaGF5bGV5c29uOTJAbQ


In [47]:
message2 = { 'summary': 'Write a draft.',
  'start': {
    'dateTime': '2023-11-28T09:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'end': {
    'dateTime': '2023-11-28T17:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  }
}
event = service.events().insert(calendarId="3669abef1c37b027d0f67733d06ea0dfdb30be790671aa54befc7679044ebebb@group.calendar.google.com", body=message2).execute()

In [ ]:
init_result = json.loads(message)

In [23]:
print(message)


{
  "summary": "Meeting with Ryan Gosling",
  "location": "",
  "description": "",
  "start": {
    "dateTime": "2023-04-21T09:00:00",
    "timeZone": "America/Los_Angeles"
  },
  "end": {
    "dateTime": "2023-04-21T10:00:00",
    "timeZone": "America/Los_Angeles"
  },
  "attendees": [
    {
      "email": "ryan.gosling@email.com"
   


In [ ]:
# 이거 내가 룰로 짜서 this monday -> 2023/11/27 이렇게 파싱하려고 했던 흔적임. 근데 그냥 chatgpt gpt4 한테 물어봤는데 잘해서 그냥 이거로 안했음..
# 그렇게 할 때의 input_text 는 이거였음 아래 참고

# input_text = """
# Detect any time-related word from the given input from the user.
# Examples include today, tomorrow, this Wednesday, next Tuesday, last Friday, 11/13, November 5th, 13th of July.
# When the detected word is day of week, make sure to include adjective in front of it such as "this", "next", "upcoming", "last", "past".
# \n\n
# Example 1
# Input: I want to schedule a meeting at 5PM tomorrow.
# Output: tomorrow

# Example 2
# Input: What time does the class start next Friday?
# Output: next Friday

# Example 3
# Input: What is the date of this upcoming Friday?
# Output: this upcoming Friday
# \n\nInput: 
# """


# query_date = None
# found_time_related_phrase = 'this Monday'
# found_time_related_phrase = found_time_related_phrase.lower()
# if found_time_related_phrase == 'today':
#     query_date = datetime.datetime.today().date()
# elif found_time_related_phrase == 'tomorrow':
#     query_date = datetime.datetime.today().date() + datetime.timedelta(days=1)
# elif found_time_related_phrase == 'yesterday':
#     query_date = datetime.datetime.today().date() - datetime.timedelta(days=1)
# elif 'day' in found_time_related_phrase:
#     phrases = found_time_related_phrase.split()
#     prefix = phrases[0]
#     dow = phrases[-1]
#     dow2num={'monday':0, 'tuesday':1, 'wednesday':2, 'thursday':3, 'friday': 4, 'saturday':5, 'sunday':6}
#     today = datetime.date.today()
#     this_dow = today + datetime.timedelta( (dow2num[dow]-today.weekday()) % 7 )
#     if prefix == 'this': # dow closest from today in the future
#         query_date = this_dow
#     elif prefix == 'next': # dow one week from this dow
#         query_date = this_dow + datetime.timedelta(7)
#     elif prefix == 'last': # dow one week prior to this dow
#         query_date = this_dow - datetime.timedelta(7)
# print(query_date)

2023-11-27


In [42]:
# to-do : 계획좀 짜주라 -> time frame ?? 이 너무 길면은 실행시간이 너무 오래 걸린다.ㅠㅠ
# to-do : 계획 중에 마일스톤은 등록하기??

input_text = f"Today is {datetime.datetime.today().date()}."
input_text += """I have NAACL deadline due at December 1st. Can you help me plan for these tasks? 
Please include some intermediate steps that I can take and create a rough schedule until the deadline. 3-hour time span is preferable.
Generate the result in a bullet point format.

Then select 3 important milestones and then generate google calendar event information for each of the the milestones.
For this, follow the JSON format below. Make sure to include 3 milestones.

Milestone 1
{ 'summary': 'Dancing in the moonlight',
  'location': 'LaLa Land',
  'description': 'I plan to dance with Ryan Gosling in the yellow dress.',
  'start': {
    'dateTime': '2023-11-28T09:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'end': {
    'dateTime': '2023-11-28T17:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'recurrence': [
    'RRULE:FREQ=DAILY;COUNT=2'
  ],
  'attendees': [
    {'email': 'ryangosling@example.com'}
  ]
}
Milestone 2
{ 'summary': 'Dancing in the moonlight',
  'start': {
    'dateTime': '2023-11-28T09:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'end': {
    'dateTime': '2023-11-28T17:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  }
}
Milestone 3
{ 'summary': 'Dancing in the moonlight',
  'location': 'LaLa Land',
  'description': 'I plan to dance with Ryan Gosling in the yellow dress.',
  'start': {
    'dateTime': '2023-11-28T09:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'end': {
    'dateTime': '2023-11-28T17:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'recurrence': [
    'RRULE:FREQ=DAILY;COUNT=2'
  ],
  'attendees': [
    {'email': 'ryangosling@example.com'}
  ]
}

"""

#  Please mark mini-deadlines or milestones with this emoji ⭐️
# {<YYYY-MM-DD>: {"9:00": <action>, "10:00": <action>, "11:00": <action>, "12:00": <action>, "13:00": <action>, "14:00": <action>, "15:00": <action>, "16:00": <action>, "17:00": <action>, "18:00": <action>, "19:00": <action>, "20:00": <action>, "21:00": <action>, "22:00": <action> },
# <YYYY-MM-DD>: {"9:00": <action>, "10:00": <action>, "11:00": <action>, "12:00": <action>, "13:00": <action>, "14:00": <action>, "15:00": <action>, "16:00": <action>, "17:00": <action>, "18:00": <action>, "19:00": <action>, "20:00": <action>, "21:00": <action>, "22:00": <action> },
# <YYYY-MM-DD>: {"9:00": <action>, "10:00": <action>, "11:00": <action>, "12:00": <action>, "13:00": <action>, "14:00": <action>, "15:00": <action>, "16:00": <action>, "17:00": <action>, "18:00": <action>, "19:00": <action>, "20:00": <action>, "21:00": <action>, "22:00": <action> }}



In [43]:


basic_chatgpt = BasicChatGPT(os.environ.get("OPENAI_API_KEY"), json_output=True, max_tokens = 1000)

basic_chatgpt.messages.append({
    "role": "user",
    "content": input_text,
})

# Call ChatGPT
response = basic_chatgpt.call()
message = response.choices[0].message.content
# init_result = json.loads(message)

print(message)
# Save the returned message
# message = response["choices"][0]["message"]
# self.messages.append(message)



In [22]:


basic_chatgpt = BasicChatGPT(os.environ.get("OPENAI_API_KEY"), json_output=True, max_tokens = 1000)

basic_chatgpt.messages.append({
    "role": "user",
    "content": input_text,
})

# Call ChatGPT
response = basic_chatgpt.call()
message = response.choices[0].message.content
# init_result = json.loads(message)

print(message)
# Save the returned message
# message = response["choices"][0]["message"]
# self.messages.append(message)



{
    "2022-11-20": {
        "9:00": "Define Hackathon project scope",
        "10:00": "Research for the Hackathon project",
        "11:00": "Break and refresh",
        "12:00": "Outline Hackathon project structure",
        "13:00": "Lunch break",
        "14:00": "Draft initial project design for Hackathon",
        "15:00": "Review progress and refine design",
        "16:00": "Set up development environment for Hackathon",
        "17:00": "Break",
        "18:00": "Start Hackathon coding (basic setup)",
        "19:00": "Dinner",
        "20:00": "Continue coding for Hackathon",
        "21:00": "End day review and plan for next day",
        "22:00": "Rest"
    },
    "2022-11-25": {
        "9:00": "Review NAACL paper progress",
        "10:00": "Write introduction for NAACL paper ⭐️",
        "11:00": "Break and refresh",
        "12:00": "Write methods section for NAACL paper",
        "13:00": "Lunch break",
        "14:00": "Continue methods section for NAACL paper",
   

In [ ]:
# 하다가 짤렸음 1000개 token을 해도 모자란 듯 ㅠ;; 대책 강구가 필요! 원래 생각은 별표 있는 것들에 대해서만 calendar 등록하려고 했었음.
# 그리고 예전에 교수님이랑 면담했을 때에는 whether needs scheduling or not 에 대한 classify를 chat gpt 가 하도록 하는걸 이야기하셨었다.
# 그것도 구현할지?? 으아..ㅠㅠ 너무 시간이 없다...........


# {
#     "2022-11-20": {
#         "9:00": "Define Hackathon project scope",
#         "10:00": "Research for the Hackathon project",
#         "11:00": "Break and refresh",
#         "12:00": "Outline Hackathon project structure",
#         "13:00": "Lunch break",
#         "14:00": "Draft initial project design for Hackathon",
#         "15:00": "Review progress and refine design",
#         "16:00": "Set up development environment for Hackathon",
#         "17:00": "Break",
#         "18:00": "Start Hackathon coding (basic setup)",
#         "19:00": "Dinner",
#         "20:00": "Continue coding for Hackathon",
#         "21:00": "End day review and plan for next day",
#         "22:00": "Rest"
#     },
#     "2022-11-25": {
#         "9:00": "Review NAACL paper progress",
#         "10:00": "Write introduction for NAACL paper ⭐️",
#         "11:00": "Break and refresh",
#         "12:00": "Write methods section for NAACL paper",
#         "13:00": "Lunch break",
#         "14:00": "Continue methods section for NAACL paper",
#         "15:00": "Review and refine methods section",
#         "16:00": "Start results section for NAACL paper",
#         "17:00": "Short refreshment",
#         "18:00": "Process data for results section",
#         "19:00": "Dinner",
#         "20:00": "Analyze first set of results for NAACL paper",
#         "21:00": "Prepare plan for final push on Hackathon project",
#         "22:00": "Rest and relax"
#     },
#     "2022-11-28": {
#         "9:00": "Prepare Hackathon workspace and resources",
#         "10:00": "Kick-off Hackathon with team",
#         "11:00": "Hackathon coding session",
#         "12:00": "Quick team status check",
#         "13:00": "Lunch break and team bonding",
#         "14:00": "Continue Hackathon coding session",
#         "15:00": "Brainstorm optimization with team",
#         "16:00": "Deploy initial Hackathon prototype for testing",
#         "17:00": "Break and refreshments",
#         "18:00": "Review prototype feedback and iterate",
#         "19:00": "Dinner with team",
#         "20:00": "Evening Hackathon coding push",
#         "21:00": "End day review with team and plan for next day",
#         "22:00": "Rest and personal downtime"
#     },
#     "2022-12-01": {
#         "9:00": "Outline final project report",
#         "10:00": "Begin writing background section of report",
#         "11:00": "Take a short break",
#         "12:00": "Define experiments and methodology for report",
#         "13:00": "Lunch break",
#         "14:00": "Write methodology section for report",
#         "15:00": "Compile results for report inclusion ⭐️",
#         "16:00": "Analyze results and create graphs/tables",
#         "17:00": "Short refreshment break",
#         "18:00": "Draft discussion section for report",
#         "19:00": "Dinner",
#         "20:00": "Proofread and review draft report",
#         "21:00": "Plan revisions and enhancements",
#         "22:00": "Well-earned rest"
#     },
#     "2022-12-04": {
#         "9:00": "Finalize Hackathon documentation",
#         "10:00": "Proofread and prepare Hackathon presentation ⭐️",
#         "11:00": "Break",
#         "12:00": "Rehearse Hackathon presentation",
#         "13:00": "Lunch break",
#         "14:00": "Prepare final submission materials for Hackathon",
#         "15:00": "Polish final project draft based on feedback",
#         "16:00": "Submit final project (if ready)",
#         "17:00": "Short break and refresh",
#         "18:00": "Resume writing NAACL paper",
#         "19:00": "Dinner",
#         "20:00": "Write abstract and conclusion for NAACL paper",
#         "21:00": "Review day's progress and adjust next steps",
#         "22:00": "End day with relaxation activities"
#     },
#     "2022-12-08": {
#         "9
